In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# variables used in script

In [3]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

### De eerste data set
Enexis decentrale opwekkingsgegevens zon-pv  2020 (1/1 + 1/7)
https://www.enexis.nl/over-ons/wat-bieden-we/andere-diensten/open-data 

In [4]:
def load_data(files):
    df = pd.DataFrame()
    for j in files:
        print('load file: ' + j)
        appenddata = pd.read_csv(j, sep=';', thousands='.', decimal=',',  encoding= 'utf-8')
        # column names hebben soms trailing spaces maar niet in beide files
        appenddata.columns = appenddata.columns.str.rstrip()
        df = df.append(appenddata,ignore_index=True)
    return df    

In [5]:
# file names
files = ['https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012020.csv'
         ,'https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01072020.csv'
        ]

In [6]:
df = load_data(files)

load file: https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01012020.csv
load file: https://opendatasitecore.blob.core.windows.net/opendata/KV_opwek/Enexis_decentrale_opwek_kv_(zon_pv)_01072020.csv


In [7]:
df.columns

Index(['Peildatum', 'Netbeheerder', 'Provincie', 'Gemeente', 'CBS Buurt',
       'CBS Buurtcode', 'Aantal aansluitingen in CBS-buurt',
       'Aantal aansluitingen met opwekinstallatie', 'Opgesteld vermogen',
       'Unnamed: 9'],
      dtype='object')

In [8]:
# check missing values
missing_total = df.isna().sum().sort_values(ascending=False)
print('Number of attributes with Null values: ' + str(len(missing_total[missing_total > 0])))
print(missing_total[missing_total > 0])

Number of attributes with Null values: 6
Unnamed: 9                           7967
CBS Buurtcode                         292
CBS Buurt                             264
Gemeente                                5
Provincie                               2
Aantal aansluitingen in CBS-buurt       1
dtype: int64


In [9]:
# vreemde kolom aan het eind
df = df.drop(columns=['Unnamed: 9'])

In [10]:
# drop alle rows waar alle columns leeg zijn
df = df.dropna(how='all') 
#df = df.dropna(subset=['Peildatum'])

In [11]:
# fill missing values 
df['CBS Buurt'] = df['CBS Buurt'].fillna('Onbekend')
df['Gemeente'] = df['Gemeente'].fillna('Onbekend')
df['Provincie'] = df['Provincie'].fillna('Onbekend')
df['Aantal aansluitingen in CBS-buurt'] = df['Aantal aansluitingen in CBS-buurt'].fillna(0)
df['CBS Buurtcode'] = df['CBS Buurtcode'].fillna(9999)

In [12]:
# select numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns  #To select all *numeric* types, use ``np.number`` or ``'number'``
#numeric_columns
# change format to integer for numeric columns
df[numeric_columns]= df[numeric_columns].astype(int)

In [13]:
# peildatum als datum (yyyy-mm)
df['Peildatum'] = pd.to_datetime(df['Peildatum'].astype(str).str[0:6], format='%Y%m', errors='ignore')

In [14]:
# fractie aansluitingen met opwekinstallatie
df['Fractie met Opwerkinstallatie'] = (df['Aantal aansluitingen met opwekinstallatie'] / df['Aantal aansluitingen in CBS-buurt']).round(2)
df['Vermogen per Opwekinstallatie in kWp'] = (df['Opgesteld vermogen'] / df['Aantal aansluitingen met opwekinstallatie']).round(1)

In [15]:
df.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,Fractie met Opwerkinstallatie,Vermogen per Opwekinstallatie in kWp
0,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000,1480,155,507,0.10,3.3
1,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-West,30001,1331,479,1685,0.36,3.5
2,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Oost,30002,2826,439,1616,0.16,3.7
3,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007,230,97,405,0.42,4.2
4,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008,102,21,61,0.21,2.9


# CBS Buurtcode koppelen aan PC4

In [16]:
file = 'https://github.com/processminery/jads_enexis/blob/main/data/pc6-gwb2020.csv?raw=true'

In [17]:
df_brt = pd.read_csv(file, sep=';', thousands='.', decimal=',',  encoding= 'utf-8')

In [18]:
df_brt.head()

,PC6,Buurt2020,Wijk2020,Gemeente2020
0,1011AB,3630400,36304,363
1,1011AC,3630400,36304,363
2,1011AD,3630400,36304,363
3,1011AE,3630400,36304,363
4,1011AG,3630403,36304,363


In [19]:
# 4 cijferige postcode uit PC6 halen
df_brt['PC4'] = df_brt['PC6'].astype(str).str[0:4]

In [20]:
df_brt.head()

,PC6,Buurt2020,Wijk2020,Gemeente2020,PC4
0,1011AB,3630400,36304,363,1011
1,1011AC,3630400,36304,363,1011
2,1011AD,3630400,36304,363,1011
3,1011AE,3630400,36304,363,1011
4,1011AG,3630403,36304,363,1011


In [21]:
# meest voorkomende PC4 per buurtcode
df2 = df_brt.groupby(['Buurt2020'])['PC4'].agg(lambda x:x.value_counts().index[0])

In [22]:
df2.head()

Buurt2020
30000    9901
30001    9903
30002    9902
30007    9902
30008    9902
Name: PC4, dtype: object

In [23]:
# join df met df2
df = pd.merge(df, df2, left_on='CBS Buurtcode', right_on='Buurt2020', how='left', suffixes=('','_2'))

In [24]:
df.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,Fractie met Opwerkinstallatie,Vermogen per Opwekinstallatie in kWp,PC4
0,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000,1480,155,507,0.10,3.3,9901
1,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-West,30001,1331,479,1685,0.36,3.5,9903
2,2020-01-01,Enexis,Groningen,Appingedam,Appingedam-Oost,30002,2826,439,1616,0.16,3.7,9902
3,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007,230,97,405,0.42,4.2,9902
4,2020-01-01,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008,102,21,61,0.21,2.9,9902
